In [1]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [2]:
# Load the movielens-100k dataset (download it if needed),
data = Dataset.load_builtin('ml-100k')

# We'll use the famous SVD algorithm.
algo = SVD()

In [3]:
data

In [4]:
from surprise import accuracy
from surprise.model_selection import train_test_split

In [5]:
trainset, testset = train_test_split(data, test_size=.25)

In [6]:
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.9429


0.9428833131941118

In [9]:
trainset

In [17]:
counts = []
for user, val in trainset.ur.items():
    counts.append(len(val))
max(counts)


557

In [18]:
len(counts)

943

In [19]:
counts = []
for user, val in trainset.ir.items():
    counts.append(user)
max(counts)

1642

In [20]:
len(counts)

1643

In [23]:
trainset.n_items

1643

In [25]:
trainset.n_users

943

In [27]:
len(algo.pu[0])

100

In [30]:
# Based on guide https://towardsdatascience.com/build-a-recommender-system-yelp-rating-prediction-example-collaborative-filtering-28a6e48a8cc

import pandas as pd
import tarfile
from tqdm import tqdm
import json
import numpy as np
import time
from copy import deepcopy

In [31]:
line_count = len(open("yelp_academic_dataset_review.json").readlines())

In [32]:
line_count

8635403

In [34]:
user_ids, business_ids, stars, dates, texts = [], [], [], [], []
with open("yelp_academic_dataset_review.json") as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        user_ids += [blob["user_id"]]
        business_ids += [blob["business_id"]]
        stars += [blob["stars"]]
        dates += [blob["date"]]
        texts += [blob["text"]]
ratings_ = pd.DataFrame(
    {"user_id": user_ids, "business_id": business_ids, "rating": stars, "date": dates, "text": texts}
)

100%|██████████| 8635403/8635403 [01:07<00:00, 128034.11it/s]


In [36]:
ratings_.head()

,user_id,business_id,rating,date,text
0,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4.0,2014-10-11 03:34:02,Apparently Prides Osteria had a rough summer a...
1,YoVfDbnISlW0f7abNQACIg,RA4V8pr014UyUbDvI-LW2A,4.0,2015-07-03 20:38:25,This store is pretty good. Not as great as Wal...
2,eC5evKn1TWDyHCyQAwguUw,_sS2LBIGNT5NQb6PD1Vtjw,5.0,2013-05-28 20:38:06,I called WVM on the recommendation of a couple...
3,SFQ1jcnGguO0LYWnbbftAA,0AzLzHfOJgL7ROwhdww2ew,2.0,2010-01-08 02:29:15,I've stayed at many Marriott and Renaissance M...
4,0kA0PAJ8QFMeveQWHFqz2A,8zehGz9jnxPqXtOc7KaJxA,4.0,2011-07-28 18:05:01,The food is always great here. The service fro...


In [37]:
active_users = user_counts.loc[user_counts >= 5].index.tolist()
ratings_ = ratings_.loc[ratings_.user_id.isin(active_users)]

NameError: name 'user_counts' is not defined

In [39]:
from collections import Counter
Counter(ratings_["user_id"])

Counter({'ak0TdVmGKo4pwqdJSTLwWw': 5,
         'YoVfDbnISlW0f7abNQACIg': 991,
         'eC5evKn1TWDyHCyQAwguUw': 1,
         'SFQ1jcnGguO0LYWnbbftAA': 4,
         '0kA0PAJ8QFMeveQWHFqz2A': 9,
         'RNm_RWkcd02Li2mKPRe7Eg': 20,
         'Q8c91v7luItVB0cMFF_mRA': 86,
         'XGkAG92TQ3MQUKGX9sLUhw': 3,
         'LWUnzwK0ILquLLZcHHE1Mw': 364,
         '99RsBrARhhx60UnAC4yDoA': 1,
         'eLAYHxHUutiXswy-CfeiUw': 3,
         'Ngl83gs3n22SzLAsNw2znw': 3,
         'hn0ZbitvmlHnF--KJGJ6_A': 26,
         'B7YSV6r1ePAXc69FkDDuZw': 3,
         'xpxWG7jQXZE6BcSeuIq4PQ': 1,
         'HvpNr0ohHCaVLp014CQrdw': 2,
         'bUHweiErUJ36WGeNrPmEbA': 35,
         'JHXQEayrDHOWGexs0dCviA': 60,
         'DECuRZwkUw8ELQZfNGef2Q': 56,
         'jySmPCkEkJR3cWJlkEs9cw': 64,
         '1RCRKuHgP3FskGUVnmFdxg': 14,
         'ltn9yaWIarK_o4DeMT1duA': 134,
         'vFnYYmtVwcMGyyGesNImVQ': 3,
         'd01SZKYmReEar7varZB0HQ': 352,
         'ffJp-ZN80M4sSkDL8Ra18w': 63,
         '7x863PREK98UuCxYpOszIw'

In [41]:
ratings_[ratings_.user_id >= 5]

TypeError: '>=' not supported between instances of 'str' and 'int'

In [43]:
v = ratings_[['user_id']]

ratings_[v.replace(v.apply(pd.Series.value_counts)).gt(5).all(1)]

KeyboardInterrupt: 

In [46]:
Counter(ratings_["user_id"][:100000])

Counter({'ak0TdVmGKo4pwqdJSTLwWw': 1,
         'YoVfDbnISlW0f7abNQACIg': 22,
         'eC5evKn1TWDyHCyQAwguUw': 1,
         'SFQ1jcnGguO0LYWnbbftAA': 1,
         '0kA0PAJ8QFMeveQWHFqz2A': 1,
         'RNm_RWkcd02Li2mKPRe7Eg': 1,
         'Q8c91v7luItVB0cMFF_mRA': 2,
         'XGkAG92TQ3MQUKGX9sLUhw': 1,
         'LWUnzwK0ILquLLZcHHE1Mw': 5,
         '99RsBrARhhx60UnAC4yDoA': 1,
         'eLAYHxHUutiXswy-CfeiUw': 1,
         'Ngl83gs3n22SzLAsNw2znw': 2,
         'hn0ZbitvmlHnF--KJGJ6_A': 2,
         'B7YSV6r1ePAXc69FkDDuZw': 1,
         'xpxWG7jQXZE6BcSeuIq4PQ': 1,
         'HvpNr0ohHCaVLp014CQrdw': 2,
         'bUHweiErUJ36WGeNrPmEbA': 2,
         'JHXQEayrDHOWGexs0dCviA': 3,
         'DECuRZwkUw8ELQZfNGef2Q': 4,
         'jySmPCkEkJR3cWJlkEs9cw': 2,
         '1RCRKuHgP3FskGUVnmFdxg': 4,
         'ltn9yaWIarK_o4DeMT1duA': 2,
         'vFnYYmtVwcMGyyGesNImVQ': 1,
         'd01SZKYmReEar7varZB0HQ': 7,
         'ffJp-ZN80M4sSkDL8Ra18w': 4,
         '7x863PREK98UuCxYpOszIw': 2,
         '9

In [58]:
counts = Counter(ratings_["user_id"])

In [48]:
df = pd.DataFrame.from_dict(counts, orient='index').reset_index()

In [49]:
df.head()

,index,0
0,ak0TdVmGKo4pwqdJSTLwWw,5
1,YoVfDbnISlW0f7abNQACIg,991
2,eC5evKn1TWDyHCyQAwguUw,1
3,SFQ1jcnGguO0LYWnbbftAA,4
4,0kA0PAJ8QFMeveQWHFqz2A,9


In [50]:
filtered_df = ratings_.join(df.set_index('index'), on='user_id')

In [51]:
len(filtered_df)

8635403

In [74]:
filt_dict = {}
users_list = []
for user, ct in counts.items():
    if ct < 5:
        users_list.append(user)
        filt_dict[user] = ct
#         print("less")
#         del counts[user]
print(len(filt_dict))

1823791


In [66]:
df = pd.DataFrame.from_dict(filt_dict, orient='index').reset_index()

In [69]:
filtered_df = ratings_.join(df.set_index('index'), on='user_id', how="inner")

In [70]:
len(filtered_df)

2868432

In [71]:
df.set_index('index')

,0
index,
SFQ1jcnGguO0LYWnbbftAA,4
XGkAG92TQ3MQUKGX9sLUhw,3
99RsBrARhhx60UnAC4yDoA,1
eLAYHxHUutiXswy-CfeiUw,3
Ngl83gs3n22SzLAsNw2znw,3
...,...
zxzgFCdxM7bX63ZVpJgk4g,1
XkNcbvhBVvEUMDDjHi2B3w,1
7-zTzm9DzYTFe5N2wkqhWw,1


In [72]:
df.set_index('index').join(ratings_.set_index('user_id'), how="inner")

,0,business_id,rating,date,text
---2PmXbF47D870stH1jqA,3,y_c38tCm3aQps72zSGW4Lw,5.0,2016-11-14 02:50:38,"What a Wonderful Evening, and what a gracious ..."
---2PmXbF47D870stH1jqA,3,i2TxS3PS3nUmhVm7i5yZYg,5.0,2015-09-28 14:06:08,What a wonderful weekend at this Hotel. It is ...
---2PmXbF47D870stH1jqA,3,WP-AaihA5jqqjCDoEywjnA,5.0,2015-10-11 20:46:33,What a wonderful weekend we had. From the mome...
---7FS-3SMy-cnnIPHcW9w,1,GzFD07Y8SbSF5ZPfhmR_ig,1.0,2019-02-25 00:09:07,My first experience with these folks was downr...
---fQxo-9tYZAkyWYrSfdA,1,zu2MOe5SZtvJUzVJBhgvpw,5.0,2020-08-31 03:51:03,This is by far my favorite café ever!! The foo...
...,...,...,...,...,...
zzzPq8j7z2qR1oYqQTJxUg,1,rT9jjRPfZC9i7OFXItaiUw,5.0,2016-05-09 17:50:38,Boston City Florist is amazing! I was in Vegas...
zzz_yXv7Z_JRT3tQmEo5rw,1,BNx5k8JjIDr-YolD320KPw,5.0,2017-09-21 13:48:11,Couldn't have been happier with my move. The s...
zzzcuxFaP_FvdIB-fbP9iA,1,KK-6rXbIo9B4-b_P5WO5Qg,3.0,2014-08-27 03:07:28,"I like writing reviews as I eat. For starters,..."
zzzl5-rnSu3jclcANGsLgg,1,hP-twYMZ5A24SUo3T4yPhg,2.0,2019-06-26 16:26:33,the fries here were ok and I sadly had to pay ...


In [77]:
filtered_df = ratings_[ratings_["user_id"].isin(users_list)]

In [80]:
ratings_ = filtered_df

In [81]:
SAMPLING_RATE = 1/5
user_id_unique = ratings_.user_id.unique()
user_id_sample = pd.DataFrame(user_id_unique, columns=['unique_user_id']) \
                    .sample(frac= SAMPLING_RATE, replace=False, random_state=1)
ratings_sample = ratings_.merge(user_id_sample, left_on='user_id', right_on='unique_user_id') \
                    .drop(['unique_user_id'], axis=1)

In [83]:
ratings_sample.head()

,user_id,business_id,rating,date,text
0,B7YSV6r1ePAXc69FkDDuZw,wZgUAuDuEGPEzKK-PsngKQ,1.0,2014-06-27 22:06:55,I wish I could give them zero stars. The call ...
1,B7YSV6r1ePAXc69FkDDuZw,5LgobuU4jEkZJhOqwGDCtg,1.0,2012-04-29 16:16:50,"I have been there three times, and there has b..."
2,B7YSV6r1ePAXc69FkDDuZw,-_bvu_oJdS363w1TYoJnYQ,5.0,2011-12-02 18:33:06,I got my hair colored and highlighted about a ...
3,vFnYYmtVwcMGyyGesNImVQ,jMz_y_-cWMfiZF7Q5snE6Q,5.0,2016-01-30 02:39:22,Ordered the original tonkotsu base ramen and a...
4,vFnYYmtVwcMGyyGesNImVQ,KedUavOevU1AXfHC_aTH9A,3.0,2016-01-29 03:25:39,"For the price we paid $28/person, I would say ..."


In [85]:
ratings_sample_cols = ratings_sample[["user_id", "business_id", "rating"]]

In [90]:
rfs = ratings_sample_cols.rename(columns={"user_id":"userID", "business_id":"itemID"})

In [91]:
rfs.head()

,userID,itemID,rating
0,B7YSV6r1ePAXc69FkDDuZw,wZgUAuDuEGPEzKK-PsngKQ,1.0
1,B7YSV6r1ePAXc69FkDDuZw,5LgobuU4jEkZJhOqwGDCtg,1.0
2,B7YSV6r1ePAXc69FkDDuZw,-_bvu_oJdS363w1TYoJnYQ,5.0
3,vFnYYmtVwcMGyyGesNImVQ,jMz_y_-cWMfiZF7Q5snE6Q,5.0
4,vFnYYmtVwcMGyyGesNImVQ,KedUavOevU1AXfHC_aTH9A,3.0


In [92]:
from surprise import Reader
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(rfs[['userID', 'itemID', 'rating']], reader)

In [93]:
trainset, testset = train_test_split(data, test_size=.25)
algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 1.5415


1.5415046408273767

In [96]:
len(algo.pu)

300949

In [101]:
len(pd.unique(rfs["userID"]))

364758

In [102]:
len(algo.qi)

99569

In [103]:
len(pd.unique(rfs["itemID"]))

110544